# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "output.csv"
csv = "output.csv"
weather_data_df = pd.read_csv("output.csv")
weather_data_df

,Unnamed: 0,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,0,dorokhovo,55.5506,36.3744,30.20,64,75,15.66,RU,1614730193
1,1,bluff,-46.6000,168.3333,73.99,47,12,11.01,NZ,1614730193
2,2,mataura,-46.1927,168.8643,73.99,47,28,11.01,NZ,1614730193
3,3,yulara,-25.2406,130.9889,82.40,34,0,14.97,AU,1614730194
4,4,hasaki,35.7333,140.8333,40.69,48,20,18.41,JP,1614730194
...,...,...,...,...,...,...,...,...,...,...
546,546,lamu,-2.2717,40.9020,76.51,86,2,9.75,KE,1614730343
547,547,laizhou,37.1717,119.9214,38.53,74,100,14.36,CN,1614730343
548,548,miles city,46.4083,-105.8406,55.40,26,1,14.97,US,1614730343
549,549,ternate,0.8000,127.4000,81.34,73,100,6.08,ID,1614730344


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

coordinates =weather_data_df[["lat", "long"]]
humidity = weather_data_df["humidity"].astype(float)

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_df = weather_data_df
# Create the names and bins for temperature
temp_bins = [-100,70, 80, 400]
temp_group_names = ["too cool", "70-80", "too hot"]

# Create the names and bins for wind
wind_bins = [-5, 10, 500]
wind_group_names = ["less than 10 mph", "more than 10 mph"]

# Create the names and bins for cloudiness
cloud_bins = [-5, 0.1, 500]
cloud_group_names = ["clear", "cloudy"]


vacation_df["temp"] = pd.cut(vacation_df["temp"], temp_bins, labels=temp_group_names, include_lowest=True)
vacation_df["wind"] = pd.cut(vacation_df["wind"], wind_bins, labels=wind_group_names, include_lowest=True)
vacation_df["cloud"] = pd.cut(vacation_df["cloud"], cloud_bins, labels=cloud_group_names, include_lowest=True)
vacation_df

,Unnamed: 0,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,0,dorokhovo,55.5506,36.3744,too cool,64,cloudy,more than 10 mph,RU,1614730193
1,1,bluff,-46.6000,168.3333,70-80,47,cloudy,more than 10 mph,NZ,1614730193
2,2,mataura,-46.1927,168.8643,70-80,47,cloudy,more than 10 mph,NZ,1614730193
3,3,yulara,-25.2406,130.9889,too hot,34,clear,more than 10 mph,AU,1614730194
4,4,hasaki,35.7333,140.8333,too cool,48,cloudy,more than 10 mph,JP,1614730194
...,...,...,...,...,...,...,...,...,...,...
546,546,lamu,-2.2717,40.9020,70-80,86,cloudy,less than 10 mph,KE,1614730343
547,547,laizhou,37.1717,119.9214,too cool,74,cloudy,more than 10 mph,CN,1614730343
548,548,miles city,46.4083,-105.8406,too cool,26,cloudy,more than 10 mph,US,1614730343
549,549,ternate,0.8000,127.4000,too hot,73,cloudy,less than 10 mph,ID,1614730344


In [6]:
ideal_vacation_df=vacation_df.loc[vacation_df["temp"]=="70-80"]
ideal_vacation_df.loc[ideal_vacation_df["wind"]=="less than 10 mph"]
hotel_df = ideal_vacation_df.loc[ideal_vacation_df["cloud"]=="clear"]
hotel_df

,Unnamed: 0,city,lat,long,temp,humidity,cloud,wind,country,date and time
63,63,saint-philippe,-21.3585,55.7679,70-80,69,clear,less than 10 mph,RE,1614730212
120,120,caborca,30.5838,-111.1010,70-80,8,clear,less than 10 mph,MX,1614730227
166,166,chabahar,25.2919,60.6430,70-80,75,clear,less than 10 mph,IR,1614730240
173,173,port blair,11.6667,92.7500,70-80,68,clear,less than 10 mph,IN,1614730242
183,183,soeng sang,14.4264,102.4606,70-80,100,clear,less than 10 mph,TH,1614730244
213,213,guerrero negro,27.9769,-114.0611,70-80,20,clear,more than 10 mph,MX,1614730252
258,258,saint-pierre,-21.3393,55.4781,70-80,64,clear,less than 10 mph,RE,1614730263
375,375,markapur,15.7333,79.2833,70-80,56,clear,less than 10 mph,IN,1614730295
439,439,buin,-33.7333,-70.7500,70-80,56,clear,more than 10 mph,CL,1614730314
535,535,mecca,21.4267,39.8261,70-80,48,clear,less than 10 mph,SA,1614730340


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel = []

# geocoordinates
target_coordinates = hotel_df["lat"],["long"]
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()
#hotel.append(response['hotel'])
print(response)

{'error_message': 'This API project is not authorized to use this API.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
